<a href="https://colab.research.google.com/github/MohamadRayi/MachineLearningTasks/blob/main/Task%20Lecture%2013%20Machine%20Learning/BAB%203/Fine_tuning_a_model_with_the_Trainer_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mohamad Rayi Dwi Putra

1103213070

Task Lecture 13

Fine_tuning_a_model_with_the_Trainer_API

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [8]:
# Mengimpor fungsi yang diperlukan dari pustaka datasets
from datasets import load_dataset

# Mengimpor kelas-kelas yang diperlukan dari pustaka transformers
from transformers import AutoTokenizer, DataCollatorWithPadding

# Memuat dataset GLUE dengan metode "mrpc"
raw_datasets = load_dataset("glue", "mrpc")

# Memilih model BERT yang akan digunakan
checkpoint = "bert-base-uncased"

# Membuat tokenizer otomatis dari model yang dipilih
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Mendefinisikan fungsi untuk tokenisasi setiap contoh dalam dataset
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Melakukan tokenisasi pada dataset menggunakan fungsi yang telah didefinisikan
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Membuat objek DataCollatorWithPadding untuk menyiapkan batch data dengan padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [9]:
# Mengimpor kelas yang diperlukan dari pustaka Transformers
from transformers import AutoModelForSequenceClassification

# Membuat objek model untuk klasifikasi urutan (sequence)
# dari model yang sudah dilatih sebelumnya (pre-trained) dengan jumlah label 2
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

In [7]:
# Mendefinisikan fungsi untuk menghitung metrik evaluasi
def compute_metrics(eval_preds):
    # Memuat metrik evaluasi dari pustaka 'evaluate' untuk tugas GLUE dan dataset MRPC
    metric = evaluate.load("glue", "mrpc")

    # Memisahkan logits (output model) dan label dari hasil evaluasi
    logits, labels = eval_preds

    # Menghasilkan prediksi dari logits dengan memilih indeks dengan nilai tertinggi
    predictions = np.argmax(logits, axis=-1)

    # Menghitung metrik menggunakan prediksi dan label
    return metric.compute(predictions=predictions, references=labels)